In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 80.9 MB/s eta 0:00:00


In [24]:
import re
import torch
import torch.nn as nn

In [13]:
train_text = list()
train_scores = list()
train_file = open('/content/en-train.txt','r')
train_text_lines = train_file.readlines()
for line in train_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-Z0-9\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-Z0-9\s]', '', s2).lower()
    train_scores.append(float(score.strip()))
    train_text.append([s1,s2])
val_text = list()
val_scores = list()
val_file = open('/content/en-val.txt','r')
val_text_lines = val_file.readlines()
for line in val_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-Z0-9\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-Z0-9\s]', '', s2).lower()
    val_scores.append(float(score.strip()))
    val_text.append([s1,s2])
test_text = list()
test_scores = list()
test_file = open('/content/test-sentences-2017.txt','r')
test_text_lines = test_file.readlines()
for line in test_text_lines:
    s1,s2,score = line.split('\t')
    s1 = re.sub(r'[^a-zA-Z0-9\s]', '', s1).lower()
    s2 = re.sub(r'[^a-zA-Z0-9\s]', '', s2).lower()
    test_scores.append(float(score.strip()))
    test_text.append([s1,s2])

In [14]:
len(train_scores),len(test_scores)

(13365, 250)

In [17]:
train_text[:10]

[['umm the california was a cargo ship i dont care what the article says',
  'the california was a cargo ship'],
 ['how can i prepare this old exterior wall for painting',
  'how do i prepare this exterior concrete wall for paint'],
 ['the man is playing the guitar', 'a man is playing guitar'],
 ['it was a final test before delivering the missile to the armed forces',
  'state radio said it was the last test before the missile was delivered to the armed forces'],
 ['this does not fully answer your question but it is certainly one consideration',
  'i am a phd student in computational science which is an interdisciplinary major spanning mathematics computer science and engineerning'],
 ['a longedhaired cat with its eyes closed',
  'a close up of a cat with its eyes closed'],
 ['the venture owns five oil refineries and more than 2100 filling stations in russia and ukraine',
  'tnkbp has six oil producing units five refineries and 2100 filling stations'],
 ['we welcome the arrangements pr

In [33]:
import torch
from transformers import BertTokenizer, BertModel

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoder = BertModel.from_pretrained('bert-base-uncased')

pred_scores = list()
for i in range(len(train_text)):
  tokens_1 = tokenizer(train_text[i][0], padding=True, truncation=True, return_tensors='pt')
  input_ids = tokens_1['input_ids']
  attention_mask = tokens_1['attention_mask']

  with torch.no_grad():
    embeddings_1 = encoder(input_ids, attention_mask=attention_mask)
    embeddings_1 = embeddings_1.last_hidden_state[:, 0, :]

  tokens_2 = tokenizer(train_text[i][1], padding=True, truncation=True, return_tensors='pt')
  input_ids = tokens_2['input_ids']
  attention_mask = tokens_2['attention_mask']

  with torch.no_grad():
    embeddings_2 = encoder(input_ids, attention_mask=attention_mask)
    embeddings_2 = embeddings_2.last_hidden_state[:, 0, :]
  
  # print(type(embeddings_1))
  cos = nn.CosineSimilarity(dim=1)
  pred_scores.append(cos(embeddings_1,embeddings_2)[0].item())
  # cos_sim = f.cosine_similarity(embeddings_1.unsqueeze(0), embeddings_2.unsqueeze(0))

  # # pred_scores.append(5*(cos_sim.item()))
  # print(cos_sim.item())


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
from scipy.stats import pearsonr
pcc, p_value = pearsonr(train_scores,pred_scores )

# Print the PCC
print("PCC: {:.3f}".format(pcc))

PCC: 0.343


In [ ]:
# import torch
# from transformers import BertTokenizer, BertModel

# # Load the BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# encoder = BertModel.from_pretrained('bert-base-uncased')

# # Define the input texts
# texts = ["Hello, how are you doing today?", "What's the weather like outside?", "The quick brown fox jumps over the lazy dog."]

# # Tokenize the texts and get the input IDs and attention masks
# tokens = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
# input_ids = tokens['input_ids']
# attention_mask = tokens['attention_mask']

# # Generate the BERT embeddings
# with torch.no_grad():
#     embeddings = encoder(input_ids, attention_mask=attention_mask)
#     embeddings = embeddings.last_hidden_state[:, 0, :]
    
# # Print the embeddings
# print(embeddings)
